# Test Modules

In [ ]:
# Set project root directory and add `src` to path
import sys
from pathlib import Path

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import the notebook setup utilities
from word2gm_fast.utils.notebook_setup import setup_testing_notebook, enable_autoreload, run_silent_subprocess

# Enable mixed precision for GPU training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Enable autoreload for development
enable_autoreload()

# Set up environment
env = setup_testing_notebook(project_root=PROJECT_ROOT)

# Extract commonly used modules for convenience
tf = env['tensorflow']
np = env['numpy']
pd = env['pandas']
print_resource_summary = env['print_resource_summary']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<pre>Autoreload enabled</pre>

<pre>Project root: /scratch/edk202/word2gm-fast
TensorFlow version: 2.19.0
Device mode: GPU-enabled</pre>

<pre>Testing environment ready!</pre>

In [27]:
print_resource_summary()

<pre>SYSTEM RESOURCE SUMMARY
============================================================
Hostname: cm045.hpc.nyu.edu

Job Allocation:
   CPUs: 28
   Memory: 250.0 GB
   Requested partitions: short,cm,cpu_a100_2
   Actually running on: SSH failed: Host key verification failed.
   Job ID: 62891081
   Node list: cm045

GPU Information:
   Error: NVML Shared Library Not Found

TensorFlow GPU Detection:
   TensorFlow detects N/A GPU(s)
   Built with CUDA: True
============================================================</pre>

In [29]:
import subprocess

result = subprocess.run([
    'pytest',
    'tests/'
], capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print(result.stderr)

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /scratch/edk202/word2gm-fast
plugins: anyio-4.9.0
collected 29 items

tests/test_corpus_to_dataset.py .                                        [  3%]
tests/test_dataset_to_triplets.py ......                                 [ 24%]
tests/test_index_vocab.py ..                                             [ 31%]
tests/test_tfrecord_io.py .......                                        [ 55%]
tests/test_word2gm_model.py s............                                [100%]

======================== 28 passed, 1 skipped in 4.06s =========================

